## Importing the Required Libraries

In [28]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
from os import listdir
from os.path import join
import cv2
import pandas as pd
import os
import random as rn
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras import layers
from tensorflow import keras
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import io

# Only enable this for tensor-core GPUs
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16")

In [29]:
SEED = 321
rn.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Hyperparameters

In [53]:
RESIZE_TO = 32
PATCH_SIZE = 32

NUM_MIXER_LAYERS = 3
HIDDEN_SIZE = 64
MLP_SEQ_DIM = 64
MLP_CHANNEL_DIM = 64

EPOCHS = 5
BATCH_SIZE = 128

# Dataset

In [31]:
data = "../input/flowers-recognition/flowers/"
folders = os.listdir(data)

In [32]:
image_names = []
train_labels = []
train_images = []

size = 32,32

for folder in folders:
    for file in os.listdir(os.path.join(data,folder)):
        if file.endswith("jpg"):
            image_names.append(os.path.join(data,folder,file))
            train_labels.append(folder)
            img = cv2.imread(os.path.join(data,folder,file))
            im = cv2.resize(img,size)
            train_images.append(im)
        else:
            continue

In [34]:
train = np.array(train_images)

train.shape

(4323, 32, 32, 3)

In [36]:
train = train.astype('float32') / 255.0

In [37]:
label_dummies = pd.get_dummies(train_labels)

labels =  label_dummies.values.argmax(1)

In [38]:
union_list = list(zip(train, labels))
rn.shuffle(union_list)
train,labels = zip(*union_list)


train = np.array(train)
labels = np.array(labels)

In [50]:
# Train, Val, Test split = 0.8, 0.1, 0.1 of the dataset
X_train,X_val,y_train,y_val = train_test_split(train,labels, test_size = 0.2)
X_val,X_test,y_val,y_test = train_test_split(X_val,y_val, test_size = 0.5)

## MLP-Mixer Utilities

In [54]:
def mlp_block(x, mlp_dim):
    x = layers.Dense(mlp_dim)(x)
    x = tf.nn.gelu(x)
    return layers.Dense(x.shape[-1])(x)

def mixer_block(x, tokens_mlp_dim, channels_mlp_dim):
    y = layers.LayerNormalization()(x)
    y = layers.Permute((2, 1))(y)
    
    token_mixing = mlp_block(y, tokens_mlp_dim)
    token_mixing = layers.Permute((2, 1))(token_mixing)
    x = layers.Add()([x, token_mixing])
    
    y = layers.LayerNormalization()(x)
    channel_mixing = mlp_block(y, channels_mlp_dim)
    output = layers.Add()([x, channel_mixing])
    return output

def mlp_mixer(x, num_blocks, patch_size, hidden_dim, 
              tokens_mlp_dim, channels_mlp_dim,
              num_classes=10):
    x = layers.Conv2D(hidden_dim, kernel_size=patch_size,
                      strides=patch_size, padding="valid")(x)
    x = layers.Reshape((x.shape[1]*x.shape[2], x.shape[3]))(x)

    for _ in range(num_blocks):
        x = mixer_block(x, tokens_mlp_dim, channels_mlp_dim)
    
    x = layers.LayerNormalization()(x)
    x = layers.Dropout(0.25)(x)
    x = layers.GlobalAveragePooling1D()(x)
    return layers.Dense(num_classes, activation="softmax", dtype="float32")(x)

In [55]:
def create_mlp_mixer():
    inputs = layers.Input(shape=(SCALE,SCALE,3))
    outputs = mlp_mixer(inputs, NUM_MIXER_LAYERS,
                        PATCH_SIZE, HIDDEN_SIZE, 
                        MLP_SEQ_DIM, MLP_CHANNEL_DIM)
    return tf.keras.Model(inputs, outputs, name="mlp_mixer")

In [56]:
mlp_mixer_classifier = create_mlp_mixer()

In [57]:
def run_experiment(model):
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    checkpoint_filepath = "/tmp/checkpoint"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=True,
    )
    
    
    history = model.fit(
        x=X_train,
        y=y_train,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_data = (X_val,y_val),
        callbacks=[checkpoint_callback],
    )

    model.load_weights(checkpoint_filepath)
    _, top_1_accuracy = model.evaluate(X_test, y_test)
    print()
    print(f"Test accuracy: {round(top_1_accuracy * 100, 2)}%")
    
    return history, model

## Model Training and Evaluation

In [58]:
history, model = run_experiment(mlp_mixer_classifier)
model.save(f"mlp_mixer_{NUM_MIXER_LAYERS}")

Epoch 1/5
28/28 [==============================] - 5s 50ms/step - loss: 1.8472 - accuracy: 0.2425 - val_loss: 1.4982 - val_accuracy: 0.3727
Epoch 2/5
28/28 [==============================] - 1s 22ms/step - loss: 1.4722 - accuracy: 0.3560 - val_loss: 1.4113 - val_accuracy: 0.3889
Epoch 3/5
28/28 [==============================] - 1s 21ms/step - loss: 1.4058 - accuracy: 0.3747 - val_loss: 1.3718 - val_accuracy: 0.4005
Epoch 4/5
28/28 [==============================] - 1s 21ms/step - loss: 1.3719 - accuracy: 0.3888 - val_loss: 1.3106 - val_accuracy: 0.4097
Epoch 5/5
14/14 [==============================] - 0s 7ms/step - loss: 1.3174 - accuracy: 0.4042

Test accuracy: 40.42%
